### And now - Week 3 Day 3

## AutoGen Core

Something a little different.

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

### The fundamental principle

Autogen Core decouples an agent's logic from how messages are delivered. (Decouples an Agent's logic from how messages are delivered)
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work. (The framework handles creation and communucation)
The Agents are responsible for their logic - that is not the remit of Autogen Core

The communication infrastructure is called a Runtime.

There are 2 types: **Standalone** and **Distributed**.

One of them is called standalone, which basically essentially means it sort of runs on your box in
a simple way. And the other distributed is that it runs in a way that could allow remote agents to interact with each
other. So these are the two kinds, and you code them both a bit differently.

Today we will use a standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.

Tomorrow we'll briefly look at a Distributed runtime.


In [ ]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)


### First we define our Message object

Whatever structure we want for messages in our Agent framework.

In [ ]:
# Let's have a simple one!
# So the first thing we do is we define our own object which is going to be used for passing information
# around the place, our own message object.
# data class means it's a class that's not going to have any feel any, any methods. All it is , is something that holds data.
# used to transport information between our agents.

# And in a way, like doing this is sort of analogous to when we were in Landgraaf.
# We started by defining a state.
# Landgraaf is a state machine.
# It's very focused on what is the state and making sure that that's something that you can move backwards
# and forwards in.
# And it's interesting that, you know, auto Jens auto gen core fundamental idea is all about messaging.
# And so the thing that you start by defining is your message.

@dataclass
class Message:
    content: str


### Now we define our Agent

A subclass of RoutedAgent. which is the typical the thing that you that you have as your as your parent as your superclass.

Every Agent has an **Agent ID** which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [ ]:
# And it's important to bear in mind that the agent that we're about to create in Autogen core is different to the agent that we just created in Agent Chat.
# Well, look, the agent that you define in Autogen core is just like a wrapper.
# It's like saying this is a thing that can be messaged, it can be created, i can be managed, it can
# be referred to and it can be messaged.
# But what you do with this is your responsibility and you're going to have to delegate to something.
# But this, this, this is like a management object, if you will, a management object which you're
# going to use.
# And it has amongst other things, it has an agent ID and every agent has a unique ID, and that ID has
# two, two parts to it, a type and a key.
# So every single agent has a type and a key.
# And that combination of type and key is then unique and can uniquely identify it.

# So this gives you a sort of good sense of the fabric of it, and that an agent class that you make is
# not actually an LLM.
# It's not an Autogen agent chat agent.
# It's just this management holder object, something which has a type and a key.
# So here is our first Autogen core agent.

class SimpleAgent(RoutedAgent):
    # Constructor
    def __init__(self) -> None:
        super().__init__("Simple")

    # Coroutine message handler
    # Any method that you decorate with message handler means that potentially this is something that will
    # receive messages and Autogen core handles.
    # The fact that you'll be able to register this, and a runtime is going to manage this.
    # And if someone sends a message to something with my name and my ID, then it's going to end up here.
    # At least it's going to end up here.
    # If what they dispatch is a message of this class.

    # So this is a bit of a pro thing, but you can have multiple different classes and you can use that to
    # be able to handle different types of message.
    # Uh this may be more detail than you need, but you you could, for example, have a text message and
    # an image message and have two separate handlers.
    # And just by virtue of the different signatures, the different method signatures, Autogen core will
    # automatically dispatch the right message to the right method, the right coroutine.

    # Uh, and so this, this simple, uh, agent is going to return a, it's going to receive a message and
    # it's going to return a message is now that I said it is a bit like this idea that in Landgraf we, uh,
    # took in our nodes, took a state and returned a state.
    # But this is just like a parallel thing.
    # But it's all about messages.

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        # Message class defined above
        # This eample isn't going to do anything with the message it receives, no LLM calls or anything like that.
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")
        

### OK let's create a Standalone runtime and register our agent type

In [ ]:
# We are going to create a runtime, a single threaded agent runtime it's called, which is a standalone
# runtime running on my computer, which as it says, will handle agents in a single threaded way.
# And the first thing that we're going to do is register.
# We call register on the agent itself to say, I want you to register yourself with this runtime.
# Now, this isn't creating an agent.
# This is just saying you are a type of agent.
# And I want you to tell the runtime that you are a type of agent that can be spawned, you can be created,
# and you are a type of agent of type simple agent that's going to be your type.
# And this, this thing here, this is a function which can generate new versions of you.
# It can instantiate you.
# It is a factory.
# And you pass that in as well.

# We haven't actually yet built any of these.
# We haven't instantiated one.
# But the as a type of agent is now a known thing to our runtime.
# Our runtime knows that there are such things as simple agents.
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

### Alright! Let's start a runtime and send a message

In [ ]:
# It is now running, its is a proper runtime
runtime.start()

In [ ]:
# And now we are going to uh, we're going to first of all create an agent ID object to identify an agent.
# And we'll we'll do this properly.
# We'll say agent ID equals that, that is an ID that would identify this.
# And we are going to say that we want the default agent.
# Uh, because, because that, that will then, uh, we'll make sure that we have an agent created,
# and then we're going to send a message to, we're going to send a message to the, to the agent, which
# is called simple agent, The default ID and we're going to send a message.
# Well hi there to it.
# And then we will print whatever comes back.

# So it's simple agent is its type, which is the same one we registered and it default is the, uh,
# the, the id uh, and it said, you said, well hi there.

# What it's there to do is to handle the passing of messages around by looking things up based on their
# type and their ID, and that's what it does.

agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

In [ ]:
await runtime.stop()
await runtime.close()

### OK Now let's do something more interesting

We'll use an AgentChat Assistant!

In [ ]:

# And we could just call OpenAI directly using OpenAI dot create.
# We could just call the Python client library.
# But we can also use Autogen agent chat and we might as well since it's autogen week.

class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        # init just just calls the superclass.
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        # And we're going to set a field called underscore delegate, which we're just sort of holding on to.
        # This is this is the the underlying.
        # This is what our agent object is going to delegate to when it actually needs some code, some, some
        # an LLM to run.
        # And we could have anything we want.
        # If you remember, when you create an assistant agent, there's the name of the agent and there's the
        # model client, and there it is.
        # And as a result in underscore delegate underscore often used to show sort of like private secret, uh,
        # variables that other people should not know about, like, like private in the, in the Java world.

        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    # Um, and that's really to sort of draw your attention to the fact that what matters is what type of
    # object comes in the message field, because Autogen core will automatically route messages to agents
    # that should receive them based on the the finding a handler that that looks for a message of that type.
    # That's how it works.
    # That's the clever routing that it does.
    # And by the way, you can send a message direct to an agent, which is what we're doing right now.
    # But it also has a whole kind of pub sub thing there where you can subscribe to topics and you can publish
    # out messages to lots of agents that are all interested in one particular topic.
    # So the whole process of dispatching messages to the right agent and then calling the right handler,
    # that's the clever stuff.

    # So anyways, handle my message type.
    # Uh, as long as, uh, message is sent to this agent with the ID and and type, and that the object
    # that's sent is of this type, it will arrive at this function.
    # And I'm going to say that I received a message and print the content.
    # Then I'm going to now now this is confusing.
    # This object here text message looks subtly different to this object here.

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        # This text message is text message from the, um, autogen.
        text_message = TextMessage(content=message.content, source="user")

        # So we create the sort of text message that you need for Autogen agent chat passing in the content of
        # our message and the source is user.
        # And this is the onmessage that we call in agent Chat land with this text message and the cancellation
        # token if you remember that thing.
        # And then we get back the content.
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)
    


In [ ]:
from autogen_core import SingleThreadedAgentRuntime
# Let's create a new runtime, a single threaded agent runtime, standalone runtime.
# We're going to register two agents the simple agent and the LM agent.

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

In [ ]:
runtime.start()  # Start processing messages in the background.
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

In [ ]:
await runtime.stop()
await runtime.close()

### OK now let's show this at work - let's have 3 agents interact!

In [ ]:
from autogen_ext.models.ollama import OllamaChatCompletionClient

# Agent 1
# Subclass of RoutedAgent
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

# Agent 2 
# Subclass of RoutedAgent
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        # Use local LLaMA model via Ollama (Can be edited to other models same as above)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [ ]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

# Agent 3
# Subclass of RoutedAgent
class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        # And it puts that together into a message.
        # It looks up the default ID, the default, uh, key for player one.
        # That type, and it looks up the default for player two, that type.
        # And it's looking them up.
        # And it then as part of handling its message, it dispatches off a message to these other agents.

        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


# So just to summarize, we have a total of three agents that we've defined here.
# Three of these like agent managers these agent wrappers, they're not they're not real agents.
# They delegate to real agents.

In [ ]:
# And quite simply, we create a threaded agent a single threaded agent runtime, which is the the simple
# kind of local runtime.=
# We register player one, we register player two and give it the ability to instantiate players one and two.
# And then we register the judge, the rock, paper, scissors, and and have everything ready to go.
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [ ]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

In [ ]:
await runtime.stop()
await runtime.close()